In [ ]:
!pip install underthesea

In [ ]:
!pip install fairseq

In [ ]:
!pip install fastbpe

In [4]:
import bs4
import time
import re
import requests
import pickle
from underthesea import word_tokenize
from transformers import RobertaForSequenceClassification, RobertaConfig
import torch
import argparse
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Conclusion

### 1. Crawl News

In [5]:
class VNExpressCrawler:
  def get_page(self, page_url):
    response = requests.get(page_url, headers={'User-Agent': 'Mozilla/5.0'})
    assert response.status_code == 200, 'Cant connect to this url'
    return bs4.BeautifulSoup(response.text, 'html.parser')

  def get_content(self, page_url):
    soup = self.get_page(page_url)

    contents = [p.text for p in soup.find('article', class_='fck_detail').find_all('p')[:-1]]
    return '\n'.join(contents)

In [6]:
crawler = VNExpressCrawler()
html = crawler.get_content('https://vnexpress.net/who-canh-bao-hau-qua-nghiem-trong-do-omicron-4396050.html')

### 2. Preprocess News

In [6]:
class Preprocessor:
  def __init__(self):
    self.stopwords = pickle.load(open('/content/drive/MyDrive/The Anh Tran/Final Coursework/Saved/stopwords.sav', 'rb'))

  def remove_html(self, text):
    return re.sub(r'<[^>]*>', '', text)

  def lower(self, text):
    return text.lower()

  def word_tokenize(self, text):
    return word_tokenize(text, format='text')

  def remove_special_chars(self, text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
    
  def remove_stopwords(self, text):
    words = text.split(' ')
    res = list()
    for word in words:
      if word not in self.stopwords:
        res.append(word)

    return ' '.join(res)

  def __call__(self, text):
    text = self.remove_html(text)
    text = self.lower(text)
    text = self.word_tokenize(text)
    text = self.remove_special_chars(text)
    text = self.remove_stopwords(text)
    return text

In [8]:
preprocessor = Preprocessor()
preprocess_news = preprocessor(html)
print(preprocess_news)

omicron số_lượng đột_biến từng một_số đột_biến liên_quan nguy_cơ thay_đổi quỹ_đạo đại_dịch tổ_chức y_tế_thế_giới who báo_cáo kỹ_thuật ban_hành hôm_nay gửi 194 quốc_gia thành_viên who tiêm vaccine nhiễm biến_chủng omicron dù tỷ_lệ nhỏ dự_đoán cần nghiên_cứu sâu hiểu rõ khả_năng omicron kháng vaccine dữ_liệu công_bố tuần đột_biến né_tránh hệ miễn_dịch khả_năng lây_truyền mạnh khả_năng lây_lan xa omicron cấp_độ toàn_cầu tùy thuộc đặc_điểm thêm làn_sóng covid 19 tương_lai gây hậu_quả nghiêm_trọng who phần đánh_giá rủi_ro báo_cáo who chưa_thể chắc_chắn mức_độ lây_truyền biến_chủng hoặc nó khả_năng né_tránh hệ miễn_dịch mức_độ bảo_vệ vaccine chống nguy_cơ lây_nhiễm diễn_tiến nặng tử_vong biến_chủng rõ bởi_vậy tổ_chức khuyến_cáo y_tế dựa thông_tin hiện_nay điều_chỉnh thêm dữ_liệu vấn_đề tổ_chức yêu_cầu quốc_gia thành_viên tăng_cường giám_sát phân_tích nắm rõ biến_chủng ncov omicron ngoài_ra who khuyến_cáo áp_dụng cách tiếp_cận dựa đánh_giá rủi_ro điều_chỉnh biện_pháp đi_lại quốc_tế một_cách k

In [9]:
len(preprocess_news.split(' '))

319

In [7]:
class Encoder:
  def __init__(self):
    # Load BPE encoder
    parser = argparse.ArgumentParser()
    parser.add_argument(
      '--bpe-codes',
      default='/content/drive/MyDrive/The Anh Tran/Final Coursework/BERT/PhoBERT_base_transformers/bpe.codes',
      required=False,
      type=str,
      help='path to fastBPE BPE'
    )

    args, unknown = parser.parse_known_args()
    self.bpe = fastBPE(args)

    # Load the dictionary
    self.vocab = Dictionary()
    self.vocab.add_from_file('/content/drive/MyDrive/The Anh Tran/Final Coursework/BERT/PhoBERT_base_transformers/dict.txt')

  def __call__(self, sent):
    subwords = '<s> ' + self.bpe.encode(sent) + ' </s>'
    encoded_sent = self.vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
    encoded_sent = [encoded_sent]
    encoded_sent = pad_sequences(encoded_sent, maxlen=256, dtype='long', value=1, truncating='post', padding='post')
    mask = [int(token_id != 1) for token_id in encoded_sent[0]]
    mask = [mask]

    return torch.tensor(encoded_sent), torch.tensor(mask)

In [11]:
encoder = Encoder()
token_ids, mask = encoder(preprocess_news)
print(token_ids)
print(mask)

tensor([[    0,  4198,  3493, 11214,   800,  4809,   150,   172,  4809,   314,
           750,   411,  6981, 18759,   116, 19795, 56408, 18239,   681,  2303,
          4050,   535,   706,  1121,  1113,   409, 20049,   257,   496,  2303,
          4050,  2796,  8627,  2002, 20536, 10631,  4198,  3493, 11214,   468,
           899,   239,  2567,   115,   410,   808,   563,   297,   259,  4198,
          3493, 11214,  5268,  8627,  1470,   576,   386,  4809,  9641,  2301,
          6930,   259, 12630,   384,   259,  6300,   604,  4198,  3493, 11214,
          5473,  1280, 32463,  1231,   187,  3981,   143,  5190,  2479,  4104,
          2158,  1195,   879,   140,  1654,   921,  2303,  4050,   230,   375,
          1928,   535,  2303,  4050,  2772,   994,  1653, 12630, 20536, 10631,
           151,   231,   259,  9641,  2301,  6930,  1653,   360,  8627,   335,
           750,  6755, 20312,   534,  1063, 20536, 10631,   297, 11966,   116,
          3120,   973,  1054,   195,   438,   947,  

### 3. Load trained model

In [12]:
# load file config
config = RobertaConfig.from_pretrained(
    '/content/drive/MyDrive/The Anh Tran/Final Coursework/Saved/Bert Fine-tune/config.json',
    from_tf=False,
    num_labels=27,
    output_hidden_states=False
)

# load model
phoBERT = RobertaForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/The Anh Tran/Final Coursework/Saved/Bert Fine-tune/pytorch_model.bin',
    config=config
)

In [13]:
phoBERT.eval()
with torch.no_grad():
  outputs = phoBERT(
    token_ids,
    token_type_ids=None,
    attention_mask=mask
  )

  logits = outputs[0]
  logits = logits.detach().numpy()

  print('Label index: ', np.argmax(logits))

Label index:  5


### 4. Conclusion

In [10]:
class VietnameseNewsClassifier:
  def __init__(self):
    self.crawler = VNExpressCrawler()
    self.preprocessor = Preprocessor()
    self.encoder = Encoder()
    self.decoder = [
                      'Âm nhạc', 'Ẩm thực', 'Bất động sản', 'Bóng đá', 'Chứng khoán', 'Cúm gà', \
                      'Cuộc sống đó đây', 'Du học', 'Du lịch', 'Đường vào WTO', 'Gia đình', \
                      'Giải trí tin học', 'Giáo dục', 'Giới tính', 'Hackers và Virus', 'Hình sự', \
                      'Không gian sống', 'Kinh doanh quốc tế', 'Làm đẹp', 'Lối sống', 'Mua sắm', \
                      'Mỹ thuật', 'Sân khấu điện ảnh', 'Sản phẩm tin học mới', 'Tennis', \
                      'Thế giới trẻ', 'Thời trang' \
                    ]

    print('Loading pretrained model ... ')
    self.config = RobertaConfig.from_pretrained(
      '/content/drive/MyDrive/The Anh Tran/Final Coursework/Saved/Bert Fine-tune/config.json',
      from_tf=False,
      num_labels=27,
      output_hidden_states=False
    )
    self.model = RobertaForSequenceClassification.from_pretrained(
      '/content/drive/MyDrive/The Anh Tran/Final Coursework/Saved/Bert Fine-tune/pytorch_model.bin',
      config=self.config
    )
    self.model.eval()
    print('Done!\n')

  def __call__(self, news_url):
    begin = time.time()
    print(f'Crawling news from url {news_url} ... ')
    start = time.time()
    content = self.crawler.get_content(news_url)
    print('Crawling finished after {:.3f} seconds!\n'.format(time.time() - start))

    print('Preprocessing news ... ')
    start = time.time()
    preprocess_news = self.preprocessor(content)
    print('Preprocessing finished after {:.3f} seconds!\n'.format(time.time() - start))

    print('Encoding news ... ')
    start = time.time()
    token_ids, mask = self.encoder(preprocess_news)
    print('Encoding finished after {:.3f} seconds!\n'.format(time.time() - start))

    print('Predicting ... ')
    start = time.time()
    outputs = self.model(
      token_ids,
      token_type_ids=None,
      attention_mask=mask
    )

    logits = outputs[0]
    logits = logits.detach().numpy()
    pred_label = self.decoder[np.argmax(logits)]
    print('Predicting finished after {:.3f} seconds!\n'.format(time.time() - start))

    print('Predicted label: ', pred_label)
    print('Time in total: {:.3f}s'.format(time.time() - begin))
    

In [11]:
classifier = VietnameseNewsClassifier()
classifier('https://vnexpress.net/ramos-hy-vong-messi-doat-qua-bong-vang-4396080.html')

Loading pretrained model ... 
Done!

Crawling news from url https://vnexpress.net/ramos-hy-vong-messi-doat-qua-bong-vang-4396080.html ... 
Crawling finished after 1.441 seconds!

Preprocessing news ... 
Preprocessing finished after 0.065 seconds!

Encoding news ... 
Encoding finished after 0.009 seconds!

Predicting ... 
Predicting finished after 1.062 seconds!

Predicted label:  Bóng đá
Time in total: 2.583s
